In [6]:
def create_best_lstm_model(best_params):
    model = Sequential()
    n_layers = best_params['lstm_n_layers']
    units = best_params['lstm_units']
    for i in range(n_layers):
        model.add(LSTM(
            units=units,
            return_sequences=(i < n_layers - 1),
            input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]) if i == 0 else None
        ))
        model.add(Dropout(rate=best_params['lstm_dropout']))
    model.add(Dense(1))
    learning_rate = best_params['lstm_learning_rate']
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model


def create_lstm_model(trial):
    model = Sequential()
    n_layers = trial.suggest_int('lstm_n_layers', 2, 4)  # Optimize the number of LSTM layers
    units = trial.suggest_categorical('lstm_units', [50, 100, 150, 200])
    for i in range(n_layers):
        model.add(LSTM(
            units=units,
            return_sequences=(i < n_layers - 1),
            input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]) if i == 0 else None
        ))
        dropout_rate = trial.suggest_categorical('lstm_dropout', [0,0.1, 0.2, 0.3, 0.4, 0.5])
        model.add(Dropout(rate=dropout_rate))  # Use specific Dropout rate
    model.add(Dense(1))
    learning_rate = trial.suggest_categorical('lstm_learning_rate', [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001])  # Optimize learning rate
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

# Define optimization objective function
def objective_LSTM(trial):
    num_samples = X_train_lstm.shape[0]*0.8
    batch_size_candidates = [16, 32, 64, 128, 256, 512,1024,2048]#
    batch_size_candidates = [bs for bs in batch_size_candidates if bs <= num_samples]
    param = {
        'epochs': trial.suggest_categorical('lstm_epochs', [10, 20, 30, 40, 50, 75, 100, 150, 200]),#
        'batch_size': trial.suggest_categorical('lstm_batch_size', batch_size_candidates)
    }
    param['batch_size'] = min(param['batch_size'], num_samples) # Ensure batch_size does not exceed the number of training samples
    model = KerasRegressor(build_fn=lambda: create_lstm_model(trial), epochs=param['epochs'], batch_size=param['batch_size'], verbose=0) #
    
    
    # Use 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mse_scores = []
    
    for train_index, val_index in kf.split(X_train_lstm):
        X_train, X_val = X_train_lstm[train_index], X_train_lstm[val_index]
        y_train_fold, y_val = y_train[train_index], y_train[val_index]
        num_samples = X_train.shape[0]
        # The 'patience' parameter defines how many epochs the model continues training without improvement before stopping
        # If 'restore_best_weights' is True, the model's weights will be restored to the point with the lowest validation loss
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        model.fit(X_train, y_train_fold, validation_data=(X_val, y_val), callbacks=[early_stopping, KerasPruningCallback(trial, 'val_loss')])
        
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        mse_scores.append(mse)
    
    return np.mean(mse_scores)


def plot_importance(df_importances, output_path, model_name, region,filename):
    
    """
    Plot bar chart and concentric pie chart of feature importance.

    Parameters:
        df_importances (DataFrame): Feature importance data, required to contain 'feature' column.
        output_path (str): Base path for output files.
        model_name (str): Model name, which will be used for folder structure and file naming.
        region (str): Region name, which will be used for output file naming.
        category_map (dict): Dictionary mapping features to categories.
    """
    # Classify features by category
    # Categories corresponding to features
    category_map = {
        'KNDVI': 'VI',
        'EVI': 'VI',
        'NDVI': 'VI',
        'modis_Gpp': 'VI',
        'Fpar': 'VI',
        'modis_LAI': 'VI',
        'PML_Gpp': 'VI',
        'Mean_Yield': 'TI',
        'Previous_Yield': 'TI',
        'year': 'TI',
        'Pre': 'MC',
        'Tmin': 'MC',
        'Solar': 'MC',
        'Tmean': 'MC',
        'Tmax': 'MC',
        'CDD': 'EC',
        'HDD': 'EC',
        'GDD': 'EC',
        'VPD': 'EC',
        'SPEI': 'EC',
        'wind_speed': 'EC',
        'SAND': 'CEC',
        'AWC': 'CEC',
        'SILT': 'CEC',
        'ORG_CARBON': 'CEC',
        'TOTAL_N': 'CEC',
        'PH_WATER': 'CEC',
        'CEC_SOIL': 'CEC',
        'CLAY': 'CEC',
        'elevation': 'CEC',
        'lat': 'CEC',
         'lon': 'CEC'
    }
    # Set file save path
    folder_path = os.path.join(output_path, '06_figure', 'results', model_name)
    os.makedirs(folder_path, exist_ok=True)

    # Calculate mean of feature contributions
    df_importances_mean = df_importances.mean(axis=1).reset_index()
    df_importances_mean['Category'] = df_importances_mean['feature'].map(category_map)
    df_importances_mean = df_importances_mean[['feature', 0, 'Category']]
    df_importances_mean.columns = ['Feature', 'Contribution', 'Category']

    # Exclude baseline feature and sort by category and contribution
    contribution_df_sorted = df_importances_mean[df_importances_mean['Feature'] != 'BASELINE']
    contribution_df_sorted = contribution_df_sorted.sort_values(by=['Category', 'Contribution'], ascending=[True, False])

    # Define category colors and gradient function
    category_colors = {
        'VI': (0.9, 0.7, 0.2, 1),  # Yellow
        'TI': (0.6, 0.3, 0.9, 1),  # Purple
        'MC': (0.7, 0.3, 0.3, 1),  # Dark red
        'EC': (0.2, 0.9, 0.9, 1),  # Cyan
        'CEC': (0.3, 0.6, 0.9, 1)  # Light blue
    }
    default_color = (0.8, 0.8, 0.8, 1)  # Gray

    def get_color_gradient(base_color, num_shades):
        gradient = np.linspace(0.4, 1, num_shades)
        return [(base_color[0], base_color[1], base_color[2], shade) for shade in gradient]

    # Inner and outer circle data
    inner_contribution = contribution_df_sorted.groupby('Category')['Contribution'].sum()
    outer_contribution = contribution_df_sorted.set_index('Feature')['Contribution']

    # Create color gradient for outer circle
    outer_colors = []
    for category in inner_contribution.index:
        category_df = contribution_df_sorted[contribution_df_sorted['Category'] == category]
        base_color = category_colors.get(category, default_color)
        gradient_colors = get_color_gradient(base_color, len(category_df))
        outer_colors.extend(gradient_colors)

    # Plot figure
    fig, ax = plt.subplots(figsize=(10, 8), dpi=1200)
    ax.set_facecolor('#f0f0f0')
    ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='gray', alpha=0.7)

    # Plot bar chart
    contribution_df_sorted = contribution_df_sorted.sort_values(by='Contribution', ascending=False)
    bar_colors = [category_colors.get(cat, default_color) for cat in contribution_df_sorted['Category']]
    ax.barh(contribution_df_sorted['Feature'], contribution_df_sorted['Contribution'], color=bar_colors)
    ax.set_xlabel('Contribution')
    ax.set_ylabel('Feature')
    ax.set_title('Feature Contributions by Category')
    ax.invert_yaxis()

    # Add legend
    handles = [plt.Rectangle((0, 0), 1, 1, color=category_colors[cat]) for cat in category_colors]
    ax.legend(handles, category_colors.keys(), loc='lower right')

    # Plot embedded concentric pie chart
    inset_ax = inset_axes(ax, width=2, height=2, loc='upper right', bbox_to_anchor=(0.8, 0.35, 0.2, 0.2), bbox_transform=ax.transAxes)
    inset_ax.pie(inner_contribution, labels=[''] * len(inner_contribution), autopct='%1.1f%%', radius=1,
                 colors=[category_colors.get(cat, default_color) for cat in inner_contribution.index], wedgeprops=dict(width=0.3, edgecolor='w'))
    inset_ax.pie(outer_contribution, labels=[''] * len(outer_contribution), radius=0.7, colors=outer_colors, wedgeprops=dict(width=0.3, edgecolor='w'))
    inset_ax.add_artist(plt.Circle((0, 0), 0.4, color='white'))

    # Save chart
    plt.savefig(filename, format='tiff', bbox_inches='tight')
    plt.show()

def plt_scatter(data,filename):
    dataframes = data[['predicts','records']]
    dataframes1 = dataframes
    r2 = r2_score(dataframes['records'], dataframes['predicts'])  
    # Calculate NRMSE  
    mse = mean_squared_error(dataframes['records'], dataframes['predicts'])  
    nrmse = np.sqrt(mse) / np.mean(dataframes['records'])  
    rrmse = calculate_rrmse1(dataframes['records'], dataframes['predicts'])
    
    mape = mean_absolute_percentage_error(dataframes['records'], dataframes['predicts'])
    acc = calculate_acc(dataframes['records'], dataframes['predicts'])
    plt.figure(figsize=(10, 6))
    plt.hexbin(dataframes['records'], dataframes['predicts'], gridsize=50, cmap='viridis', mincnt=1)
    cb = plt.colorbar(label='Density')
    fit_params = np.polyfit(dataframes['records'], dataframes['predicts'], 1)
    fit_line = np.polyval(fit_params, dataframes['records'])
    plt.plot(dataframes['records'], fit_line, color='red', label='Fit line')# Add 1:1 line
    max_val = max(max(dataframes['records']), max(dataframes['predicts']))
    plt.plot([0, max_val], [0, max_val], linestyle='--', color='green', label='1:1 line')
    plt.title('')
    plt.xlabel('records')
    plt.ylabel('predicts')
    plt.grid(True)
    plt.text(0.02, 0.95, f'R² = {r2:.2f}', transform=plt.gca().transAxes, fontsize=12, va='top')  
    plt.text(0.02, 0.90, f'ACC = {acc:.2f}', transform=plt.gca().transAxes, fontsize=12, va='top')  
    plt.text(0.02, 0.85, f'RRMSE = {rrmse:.2f}%', transform=plt.gca().transAxes, fontsize=12, va='top')  
    plt.text(0.02, 0.80, f'MAPE = {mape*100:.2f}%', transform=plt.gca().transAxes, fontsize=12, va='top')  
    
    # Display the figure  
    plt.savefig(filename)
    plt.show()

In [1]:
import os
import sys
import ast
root_directory = os.getcwd()[0:3] 
sys.path.append(root_directory+'SCI\\SCI9\\2_DataAnalysis\\04_code\\1_04_Soybean')
sys.path.append(r'C:\ProgramData\anaconda3\Lib\site-packages') 
sys.path.append(r'C:\Users\DELL\.conda\envs\myenv\Lib\site-packages') 
sys.path.append(r'C:\Users\DELL\.conda\envs\rasterio_env\Lib\site-packages') 
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.preprocessing import StandardScaler
from functions import prepare_and_model_data,extract_selected_variables
from functions import calculate_rrmse1,calculate_rrmse2,calculate_acc
from sklearn.metrics import r2_score


import optuna
from optuna_integration.keras import KerasPruningCallback
from optuna.samplers import TPESampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_percentage_error
from functions import calculate_rrmse1,calculate_rrmse2,calculate_acc,calculate_nrmse
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from sklearn.model_selection import train_test_split
from IPython.display import display
from tqdm.notebook import tqdm
import tensorflow.keras.backend as K
import json

# 需要修改下
countryID = '06_India';
country =countryID[3:]
crop = '02_wheat';yield_type= 'actual_yield';
inputpath_base = root_directory + '\\SCI\\SCI9_1\\02_data\\'+crop+'\\'+countryID
root_directory = os.getcwd()[0:3]
inpath_dates_other = root_directory + '\\SCI\\SCI9_1\\02_data\\'+crop+'\\'+countryID+'\\'+'01_data'+'\\'+'07_Information'
other_infornamtion = pd.read_csv(os.path.join(inpath_dates_other,'information.txt'), sep=' ', header=None)
startyear,endyear,shp_name = other_infornamtion.iloc[0,0],other_infornamtion.iloc[0,1],other_infornamtion.iloc[0,2]
regions = ['I']
Forecastyear = endyear
# 定义每个区的最佳预报年
Forecastyears = {
    'I': Forecastyear}


years = range(startyear,endyear+1)
years_str = [str(year) for year in years]
modelname = 'LSTM'
yield_type = 'actual_yield';
SelFeature_infornamtion = extract_selected_variables(inputpath_base)
institution = 'ECMWF';

C:\Users\DELL\.conda\envs\myenv\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [8]:
for region in regions:
    ################### Load Selected Feature Variables #########################################
    Forecastyear = Forecastyears[region]
    data= pd.read_csv(os.path.join(inputpath_base, '01_data','05_buildmodel','03_modeldata',region+'_data_ori.csv'))
    # data= pd.read_csv(os.path.join(inputpath_base, '01_data','05_buildmodel','03_modeldata',region+'_data_ori.csv'))
    data = data.drop_duplicates(subset=['year', 'idJoin'], keep='first')
    TimeFeatures_sel, Static_sel, regionID = SelFeature_infornamtion[SelFeature_infornamtion['regionID'] == region].iloc[0]
    feature_all = TimeFeatures_sel+Static_sel
    filtered_columns = [col for col in data.columns if any(feature in col for feature in feature_all)]
    filtered_columns = [col for col in filtered_columns if 'year.1' not in col] # Try excluding yield-related features

    
    #### Exclude yield data from previous years (check if accuracy changes) #########################
    filtered_columns = [col for col in filtered_columns if 'Yield' not in col] # Try excluding yield-related features
    Static_sel= [col for col in Static_sel if 'Yield' not in col] # Try excluding yield-related features
    feature_all = [col for col in feature_all if 'Yield' not in col] # Try excluding yield-related features
    #############################################################
    
    MLdata_reduced = data[filtered_columns+[yield_type]]
    MLdata_reduced['year'] = data['year']
    
    # Load selected weeks
    inpath_dates = os.path.join(inputpath_base, '01_data','05_buildmodel', '02_extractdates','gs_three_periods.txt')
    gs_infornamtion = pd.read_csv(inpath_dates, delim_whitespace=True, header=None)
    gs_infornamtion.columns = ['start_point', 'peak', 'harvest_point', 'VI_select2','regionID']
    start_point, peak, harvest_point, VI_select2, region = gs_infornamtion[gs_infornamtion['regionID'] == region].iloc[0]
    
    ############################ Data Standardization ###########################################################
    data_all = MLdata_reduced;X_all = data_all.drop([yield_type], axis=1);
    
    y_all = data_all[yield_type];
    # Data standardization
    scaler_X = StandardScaler().fit(X_all)
    scaler_y = StandardScaler().fit(y_all.values.reshape(-1, 1))

    
    dataframes = pd.DataFrame()
    df_importances = pd.DataFrame()
    os.makedirs(os.path.join(inputpath_base,  '03_results',modelname,yield_type,region),exist_ok=True)
    data_all = MLdata_reduced[(MLdata_reduced['year'] < Forecastyear)]
    X_all = data_all.drop([yield_type], axis=1)
    y_all = data_all[yield_type]
    X = scaler_X.transform(X_all)
    y = scaler_y.transform(y_all.values.reshape(-1, 1)).flatten()
    X = pd.DataFrame(data=X,columns=X_all.columns.tolist())
    if start_point>harvest_point:
        weeks_select_list= list(range(start_point, 47))+list(range(1, harvest_point+1))
    else:
        weeks_select_list= list(range(start_point,harvest_point+1))
        
    data_list = []
    for i in weeks_select_list:
        data_i = X[[f'Week{i}_{feature}' for feature in TimeFeatures_sel] + Static_sel]
        data_list.append(data_i.values)
    
    # Stack all arrays in data_list into a 3D array
    data_list = np.array(data_list)
    X_train_lstm = np.transpose(data_list, (1, 0, 2)) # Reshape array to (samples, time steps, features)
    y_train = y
    
    data_test =  MLdata_reduced[(MLdata_reduced['year'] == Forecastyear)]
    X_test = data_test.drop([yield_type], axis=1);y_test = data_test[yield_type]
    X_test = scaler_X.transform(X_test)
    y_test = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()
    data_list = []
    
    X_test = pd.DataFrame(data=X_test,columns=X_all.columns.tolist())
    for i in weeks_select_list:
        data_i = X_test[[f'Week{i}_{feature}' for feature in TimeFeatures_sel] + Static_sel]
        data_list.append(data_i.values)
    data_list = np.array(data_list)
    X_test_lstm = np.transpose(data_list, (1, 0, 2)) # Reshape array to (samples, time steps, features)
    # Load saved optimal hyperparameters from JSON file and build model
    with open(f"{country}_{modelname}_region{region}_best_params.json", "r") as f:
        best_params = json.load(f)
        
    model = create_best_lstm_model(best_params)#study.best_params
    early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

    ## Train and save the model

    # Randomly split into 8:2 for training and validation data
    x_train_lstm,x_val_lstm,y_train,y_val = train_test_split(X_train_lstm, y_train, test_size=0.2, random_state=42)
    model.fit(x_train_lstm, y_train,validation_data=(x_val_lstm,y_val),epochs=best_params['lstm_epochs'], batch_size=best_params['lstm_batch_size'], callbacks=[early_stopping])

    # Save trained model
    outpathmodel = os.path.join(inputpath_base,'04_model',region, 'LSTM')
    os.makedirs(outpathmodel,exist_ok=True)
    model_path = os.path.join(outpathmodel,region+'my_lstm_model.keras')
    model.save(model_path)

    # Or load trained model
    outpathmodel = os.path.join(inputpath_base,'04_model',region, 'LSTM')
    model_path = os.path.join(outpathmodel,region+'my_lstm_model.keras')
    model = load_model(model_path)

    # Validate accuracy on test (forecast year) data 
    y_pred = model.predict(X_test_lstm)
    y_pred_inverse = scaler_y.inverse_transform(y_pred)
    y_test_inverse = scaler_y.inverse_transform(y_test.reshape(-1, 1))
    data_pre = pd.DataFrame({'predicts':y_pred_inverse.ravel(),'records':y_test_inverse.ravel()})
    data_pre['year'] = Forecastyear
    
    # Save and output results
    os.makedirs(os.path.join(inputpath_base, '03_results',modelname,region), exist_ok=True)
    data_pre.to_csv(os.path.join(inputpath_base,  '03_results',modelname,region,str(Forecastyear)+'.csv'))
    df_importances.to_csv(os.path.join(inputpath_base,  '03_results',modelname,region,f'lstm_importance_all in {region}.csv'))
    
     ############# Compare if the prediction model accuracy is better than ZeroR algorithm #######################################
    Test_year = Forecastyear
    data1 = data[['year','actual_yield','idJoin']].pivot(index='idJoin', columns='year', values='actual_yield')
    train_df = data1.iloc[:,0:Forecastyear-startyear]
    test_df = data1.loc[:,Forecastyear]
    
    train_means = train_df.mean(axis=1)
    combined = pd.concat([train_means.reset_index(drop=True), test_df.reset_index(drop=True)], axis=1)
    combined_cleaned = combined.dropna()  # Remove any rows containing NaN
    combined_cleaned.columns = ['predicts','records']
    
    folder_path =  os.path.join(inputpath_base,  '06_figure2','results', modelname,yield_type)
    os.makedirs(folder_path, exist_ok=True)
    
    filename = os.path.join(folder_path,f'null model scatter in {region} in {Forecastyear}.tiff')
    plt_scatter(combined_cleaned,filename) # Plot prediction scatter plot
    # Plot feature importance using permutation method
    
    filename = os.path.join(folder_path,f'prediction scatter in {region} in {Forecastyear}.tiff')
    plt_scatter(data_pre,filename) # Plot prediction scatter plot


    # Permutation Feature Importance（PFI）计算因子重要性
    oof_preds = model.predict(x_val_lstm, verbose=0).squeeze() 
    baseline_mae = np.mean(np.abs(oof_preds-y_val))
    results.append({'feature':'BASELINE','mae':baseline_mae}) 
    results=[]# Store feature importance ranking
    for k in tqdm(range(len(feature_all))):
        # SHUFFLE FEATURE K
        save_col = x_val_lstm[:,:,k].copy()
        np.random.shuffle(x_val_lstm[:,:,k])      
        # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
        oof_preds = model.predict(x_val_lstm, verbose=0).squeeze() 
        mae = np.mean(np.abs(oof_preds-y_val))
        results.append({'feature':feature_all[k],'mae':mae})
        x_val_lstm[:,:,k] = save_col
    df = pd.DataFrame(results)
    df = df.sort_values('mae')
    df.to_csv(os.path.join(inputpath_base,  '03_results',modelname,yield_type,region,f'lstm_importance{Forecastyear}.csv'), index=False)
    df.rename(columns={'mae':Forecastyear}, inplace=True)
    df.set_index('feature', inplace=True)
    df_importances = pd.concat([df, df_importances], axis=1)
    filename = os.path.join(folder_path,f'importance in {region} in {Forecastyear}.tiff')
    plot_importance(df_importances, inputpath_base, modelname, region,filename) 
    # Plot feature importance using permutation method

In [15]:
'''
# 2. Extract specific hyperparameter values

model_path = os.path.join(outpathmodel,region+'my_lstm_model.keras')
model = load_model(model_path)
print("\n" + "="*50)
print("Hyperparameter settings:")
print("="*50)

# Get LSTM layers and Dropout layers
lstm_layers = [l for l in model.layers if 'lstm' in l.name]
dropout_layers = [l for l in model.layers if 'dropout' in l.name]

# Print key parameters
print(f"• Number of LSTM layers: {len(lstm_layers)}")
print(f"• Number of neurons per layer: {lstm_layers[0].units} (same for all layers)")

if dropout_layers:
    print(f"• Dropout rate: {dropout_layers[0].rate:.1f} (same for all layers)")
else:
    print("• Dropout rate: not used")

# 3. Get optimizer parameters
optimizer = model.optimizer
if optimizer:
    print(f"• Optimizer type: {optimizer.__class__.__name__}")
    print(f"• Learning rate: {optimizer.learning_rate.numpy():.6f}")
else:
    print("• Optimizer information: not saved")

# 4. Get input/output dimensions
input_shape = model.input_shape[1:]
output_shape = model.output_shape[1:]
print(f"\n• Input dimensions: {input_shape}")
print(f"• Output dimensions: {output_shape}")
'''


超参数具体设置:
• LSTM 层数: 3
• 每层神经元数: 100 (所有层相同)
• Dropout 丢弃率: 0.2 (所有层相同)
• 优化器类型: Adam
• 学习率: 0.001000

• 输入维度: (19, 17)
• 输出维度: (1,)


In [ ]:
# Parameter Optimization
'''
storage_name =  f"sqlite:///{country}_{country}_{modelname}_region{region}_1007.db"
study_name = f"{country}_{modelname}_region{region}_1007"
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(), study_name=study_name, storage=storage_name, load_if_exists=True)
study.optimize(objective_LSTM, n_trials=100, n_jobs=4)  # Use 16-core CPU for parallel computing
# Output best hyperparameters
print('Best parameters: ', study.best_params)
print('Best MSE: ', study.best_value)

# Load existing study
import matplotlib.pyplot as plt
optuna.visualization.plot_param_importances(study)
plt.show()
# Visualize convergence
df = study.trials_dataframe()
plt.figure(figsize=(12, 6))
plt.plot(df['number'], df['value'])
plt.xlabel('Trial')
plt.ylabel('MSE')
plt.title('Convergence Plot')
plt.grid(True)
plt.show()
'''

[I 2024-12-06 16:32:56,442] A new study created in RDB with name: China_LSTM_regionII_1007
